In [36]:
# How to add chat history in Conversional Question & Answer

# https://python.langchain.com/v0.2/docs/how_to/qa_chat_history_how_to/


import os
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
import bs4
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter


#  chains as runnables



# loading env varables
from dotenv import load_dotenv
load_dotenv()



# load environment variables
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


# llm model
llm = ChatGroq(model="llama-3.1-8b-instant")

# embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",show_progress= True)



loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# st.write(docs)




In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever()


# vector_store = Chroma(
#     collection_name="embedding_collection",
#     embedding_function=embeddings,
#     persist_directory="./chroma_db",  # Where to save data locally, remove if not necessary
# )
# vector_store.add_documents(documents=docs)
# retriever = vectorstore.as_retriever()

Batches: 100%|██████████| 2/2 [00:06<00:00,  3.11s/it]


In [4]:
len(vectorstore.get()["ids"])

63

In [38]:



### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)



history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)




### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [39]:

### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [40]:
conversational_rag_chain.invoke(
    {"input": "what is  Agent System Overview?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

Batches: 100%|██████████| 1/1 [00:00<00:00, 20.43it/s]


'The Agent System Overview refers to a high-level description of a LLM-powered autonomous agent system, which consists of several key components, including:\n\n1. Planning: breaking down large tasks into smaller subgoals and handling complex tasks efficiently.\n2. Reflection and refinement: self-criticism, self-reflection, learning from mistakes, and refining past actions for future steps.\n\nThese components work together to enable the agent to perform tasks, solve problems, and improve its performance over time.'

In [41]:
conversational_rag_chain.invoke(
    {"input": "how we do this?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]
# message = retriever.invoke("what is  Agent System Overview?")
# contextualize_q_prompt.invoke({"input":"what is Agent System Overview?","context":message})

Batches: 100%|██████████| 1/1 [00:00<00:00,  9.00it/s]


'The key components involved in the Agent System Overview are:\n\n1. Planning: can be done through subgoal and decomposition, reflection and refinement, and task-specific instructions.\n2. Memory: can be managed through task decomposition, human inputs, or using an external classical planner.\n3. Planning & Reacting: translating reflections and environment information into actions.\n\nExamples of prompts to achieve these components include:\n\n- Subgoal and decomposition: "Steps for XYZ.\\n1."\n- Reflection and refinement: asking the LLM to generate high-level questions and answer them.\n- Memory management: using task-specific instructions or human inputs.\n- Planning & Reacting: translating the reflections and environment information into actions.'

In [ ]:
# combine_docs_chain = create_stuff_documents_chain(llm, contextualize_q_prompt)
# retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain) 
# result = retrieval_chain.invoke({"input":"what is Agent System Overview?"})
# print(result["answer"])

Batches: 100%|██████████| 1/1 [00:00<00:00, 28.14it/s]


In [33]:
# from langchain_core.runnables import (
#             RunnableParallel,
#         )

# chain =  RunnableParallel({"input":RunnablePassthrough(),"context": retriever }) | contextualize_q_prompt | llm | StrOutputParser()
# print(chain.invoke("what is Agent System Overview?"))

In [48]:
print(store["abc123"])

Human: what is  Agent System Overview?
AI: The Agent System Overview refers to a high-level description of a LLM-powered autonomous agent system, which consists of several key components, including:

1. Planning: breaking down large tasks into smaller subgoals and handling complex tasks efficiently.
2. Reflection and refinement: self-criticism, self-reflection, learning from mistakes, and refining past actions for future steps.

These components work together to enable the agent to perform tasks, solve problems, and improve its performance over time.
Human: how we do this?
AI: The key components involved in the Agent System Overview are:

1. Planning: can be done through subgoal and decomposition, reflection and refinement, and task-specific instructions.
2. Memory: can be managed through task decomposition, human inputs, or using an external classical planner.
3. Planning & Reacting: translating reflections and environment information into actions.

Examples of prompts to achieve thes